In [1]:
import mlrun
import time

In [2]:
project_name = "model-serving"
project = mlrun.get_or_create_project(name=project_name)

In [3]:
# mlrun: start-code

In [4]:
import mlrun

In [5]:
preprocess_seconds = 0.1
prediction_seconds = 0.1
postprocess_seconds = 0.1

In [6]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        pass

    def predict(self, body: dict):
        # time.sleep(prediction_seconds)
        return []


#     def preprocess(self, request: dict, operation) -> dict:
#         time.sleep(preprocess_seconds)
#         return request

#     def postprocess(self, request: dict):
#         time.sleep(postprocess_seconds)
#         return request

In [7]:
# mlrun: end-code

In [8]:
fn = mlrun.code_to_function(
    "v2-model-server",
    kind="serving",
    code_output=".",
    image="mlrun/mlrun:1.7.0-rc6",
)
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.spec.default_class = "ClassifierModel"

In [10]:
fn.add_model("mymodel", model_path="abc")

In [ ]:
fn.apply(mlrun.mount_v3io())
invoke_url = fn.deploy(project=project_name)

In [ ]:
!wget https://github.com/gtopper/postto/releases/download/v1.2.1/postto_v1.2.1_linux_amd64

In [ ]:
!chmod u+x postto_v1.2.1_linux_amd64

In [ ]:
import os

os.environ["BENCH_INVOKE_URL"] = invoke_url
os.environ["BENCH_CONCURRENT_REQUESTS"] = "16"

In [ ]:
!for i in {1..50000}; do echo '{"inputs": []}'; done | ./postto_v1.2.1_linux_amd64 -num-concurrent-requests $BENCH_CONCURRENT_REQUESTS $BENCH_INVOKE_URL